In [315]:
#Brker CNN arkitektur kombinert med PyTorch 


In [316]:
import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd

#other libraries
# from tqdm import tqdm
# import time
# import random
import os
import sys
from pathlib import Path
import h5py

#torch specific
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

In [317]:
module_path = str(Path.cwd().parents[0].parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "bh" / "BH_n5_M8_res50_10000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "sph" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_10000_events.h5")

In [318]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

In [319]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [320]:
bhTensor = torch.from_numpy(bhArray)
sphTensor = torch.from_numpy(sphArray)

In [321]:
class CalorimeterDataset(data.Dataset):
    """
    Creates dataset with histogram images and labels
    """
    def __init__(self, images, labels, transform=None):
            self.img_labels = labels
            self.images = images
            self.transform = transform
            
            
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.img_labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [322]:
labels = torch.cat((torch.from_numpy(np.zeros(np.size(bhArray))),torch.from_numpy(np.ones(np.size(sphArray)))),0)

In [323]:
data = torch.cat((bhTensor,sphTensor),0)

In [324]:
dataset = DataLoader(CalorimeterDataset(data,labels), shuffle=True, batch_size=50)

In [325]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(50*50*3,2,10)
    
    def forward(self,x):
        x = self.pool(F.relu(self.conv1(x)))
        return x

net = Net()

In [326]:
critation = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

In [328]:
dataset.batch_size

50

In [330]:
i = 0
for epoch in range(1):
    running_loss = 0.0
    for data in dataset:
        x, y = data
        i = i+1

        optimizer.zero_grad()

        outputs = net(x)
        loss = critation(x, y)
        loss.backward()
        optimizer.step() 
        
           


IndexError: index 130683354 is out of bounds for dimension 0 with size 20000